<b>Data mining Project - 2021/22</b><br/>
<span>
<b>Authors:</b> Mariagiovanna Rotundo (560765), Nunzio Lopardo (600005)</a> and Renato Eschini (203021)<br/>
<b>Group:</b>3<br/>
<b>Release date:</b> 26/12/2021
</span>

# Data Preparation

In this notebook we modify the tennis dataset according observation done in the understanding phase and we create a dataset of players

**Import libraries**

In [ ]:
import math
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

import collections
from scipy.stats.stats import pearsonr
import pandas as pd
import os
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta

import seaborn as sns
import re

In [ ]:
#import functions from ipynb file
from ipynb.fs.full.functions_understanding import *

**Loading datasets**

In [ ]:
# load of the data
DATASET_DIR = "dataset" + os.path.sep
df_tennis = pd.read_csv(DATASET_DIR + 'tennis_matches.csv', sep=',', index_col=0) 

#index_col=False say to not use the first column as ID
df_male = pd.read_csv(DATASET_DIR + 'male_players.csv', sep=',', index_col=False)
df_female = pd.read_csv(DATASET_DIR + 'female_players.csv', sep=',', index_col=False) 

#### Remove rows that have only null values

In [ ]:
#remove null rows
df_male.dropna(how='all', inplace=True)
df_female.dropna(how='all', inplace=True)
df_tennis.dropna(how='all', inplace=True)

## Male and female datasets

**romove duplicate rows in male and female dataset**

In [ ]:
#remove duplicates in the separate datasets
df_male.drop_duplicates(keep='first', inplace=True)
df_female.drop_duplicates(keep='first', inplace=True)

#### male: correction invalid names/surnames

The null values in name/surname are replaced with the value "unknown"

In [ ]:
#rows with nulls in name: replace null with unknown
df_male.loc[df_male['name'].isna(), ['name']]='unknown'
df_male['name'].isna().sum(axis = 0)

In [ ]:
#rows with nulls in surname: replace null with unknown
df_male.loc[df_male['surname'].isna(), ['surname']]='unknown'
df_male['surname'].isna().sum(axis = 0)

If a row is read in a wrong way by pandas putting all in the name, the row is splitted in name and surname updating both. The name and surname are split looking at "," that is the separator used in the dataset.

In [ ]:
#errors reading from file: all in name and null surname (replaced with unknown)
df_male[df_male['name'].str.contains(",")]
for index, row in df_male[df_male['name'].str.contains(",")].iterrows():
    if row['surname']=='unknown':
        name = row['name'].split(",")
        df_male.at[index,'surname']=name[len(name)-1]
        df_male.at[index,'name']=' '.join(name[0:len(name)-1])
        
#print(df_male.loc[40071])

Correction of invalid characters in name and surname columns. If the value has not alphabetich characters the value is considered as "unknown". If in the name there are numbers or special characters different from ",", " ' ", "-", "\_", ".", these are removed. if in the surname there are strings between "()" these are removed. All the other not alphabetic characters are replaced with a space and double spaces are removed. This checks are done for both male and female dataset to find duplicate players with invalid characters between characters.

In [ ]:
#names/surnames without any letters are invalid: set to unknown
df_male.loc[~df_male['name'].str.contains("[a-zA-Z]"), ['name']]='unknown'
df_male.loc[~df_male['surname'].str.contains("[a-zA-Z]"), ['surname']]='unknown'

#replace numbers in names
df_male["name"] = df_male["name"].str.replace("[0-9]","",regex=True)
df_male["surname"] = df_male["surname"].str.replace("[0-9]","",regex=True)

#characters not indicated in the expression are considered invalid
df_male["name"] = df_male["name"].str.replace("[^a-zA-Z ',-_.]","",regex=True)
#for surname remove string between ()
for index, row in  df_male[df_male['surname'].str.contains('\(')].iterrows():
    pos = df_male.at[index,'surname'].find("(")
    df_male.at[index,'surname']= df_male.at[index,'surname'][0:pos]
df_male["surname"] = df_male["surname"].str.replace("[^a-zA-Z ',-._]","",regex=True)

#replace allowed 
df_male["name"] = df_male["name"].str.replace("[^a-zA-Z ]"," ",regex=True)
df_male["surname"] = df_male["surname"].str.replace("[^a-zA-Z ]"," ",regex=True)

#trim apply
df_male["name"] = df_male["name"].str.strip()
df_male["surname"] = df_male["surname"].str.strip()

After these changes we look for new duplicates introduced because of changes. Then we romove rows with both name and surname unknown.

In [ ]:
#check if duplicates are been introduced
df_male.duplicated(keep='first').sum()

In [ ]:
#remove duplicates
df_male.drop_duplicates(keep='first', inplace=True)

In [ ]:
#remove (if there is) the row <unknown unknown>
row = df_male.index[(df_male['name'].str.lower()=='unknown') & (df_male['surname'].str.lower()=='unknown')].tolist()
df_male.drop(row, inplace=True)

#### female: correction invalid names/surnames

For the female dataset the same checks for the male one are done

In [ ]:
#rows with nulls in name: replace null with unknown
df_female.loc[df_female['name'].isna(), ['name']]='unknown'
df_female['name'].isna().sum(axis = 0)

In [ ]:
#rows with nulls in surname: replace null with unknown
df_female.loc[df_female['surname'].isna(), ['surname']]='unknown'
df_female['surname'].isna().sum(axis = 0)

In [ ]:
#errors reading from file: all in name and null surname (replaced with unknown)
df_female[df_female['name'].str.contains(",")]
for index, row in df_female[df_female['name'].str.contains(",")].iterrows():
    if row['surname']=='unknown':
        name = row['name'].split(",")
        df_female.at[index,'surname']=name[len(name)-1]
        df_female.at[index,'name']=' '.join(name[0:len(name)-1])

In [ ]:
#names/surnames without any letters are invalid: set to unknown
df_female.loc[~df_female['name'].str.contains("[a-zA-Z]"), ['name']]='unknown'
df_female.loc[~df_female['surname'].str.contains("[a-zA-Z]"), ['surname']]='unknown'

#replace numbers in names
df_female["name"] = df_female["name"].str.replace("[0-9]","",regex=True)
df_female["surname"] = df_female["surname"].str.replace("[0-9]","",regex=True)

#characters not indicated in the expression are considered invalid
df_female["name"] = df_female["name"].str.replace("[^a-zA-Z ',-_.]","",regex=True)
#for surname remove string between ()
for index, row in  df_female[df_female['surname'].str.contains('\(')].iterrows():
    pos = df_female.at[index,'surname'].find("(")
    df_female.at[index,'surname']= df_female.at[index,'surname'][0:pos]
df_female["surname"] = df_female["surname"].str.replace("[^a-zA-Z ',-._]","",regex=True)

#replace allowed 
df_female["name"] = df_female["name"].str.replace("[^a-zA-Z ]"," ",regex=True)
df_female["surname"] = df_female["surname"].str.replace("[^a-zA-Z ]"," ",regex=True)

#trim apply
df_female["name"] = df_female["name"].str.strip()
df_female["surname"] = df_female["surname"].str.strip()

In [ ]:
#check if duplicates are been introduced
df_female.duplicated(keep='first').sum()

In [ ]:
#remove duplicates
df_female.drop_duplicates(keep='first', inplace=True)

In [ ]:
#remove (if there is) the row <unknown unknown>
row = df_female.index[(df_female['name'].str.lower()=='unknown') & (df_female['surname'].str.lower()=='unknown')].tolist()
df_female.drop(row, inplace=True)

#### Player dataset

Creation of a dataset of players. A row for each player (name + surname) is created. Creation of columns of name (name + surname) and sex retrieved by datasets male and female looking in which of the 2 the player is. If a player is in both male and female dataset the sex in considered unknown ("U"). If name or surname are unknown only the value in the other column is considered.

In [ ]:
df_male['whole_name'] = df_male['name']+" "+df_male['surname']
df_male['sex']='M'
df_female['whole_name'] = df_female['name']+" "+df_female['surname']
df_female['sex']='F'

In [ ]:
#creation of the dataset of the player including the information about sex
df_players = pd.concat([df_male, df_female])

In [ ]:
#see if a name can be both male and female and manage them
df_players.loc[df_players.duplicated(subset=['name','surname'],keep=False)==True, ['sex']]='U'
df_players.drop_duplicates(keep='first', inplace=True)
df_players

In [ ]:
for index, row in df_players[df_players['name']=='unknown'].iterrows():
    df_players.loc[index, 'whole_name'] = row['surname']

In [ ]:
for index, row in df_players[df_players['surname']=='unknown'].iterrows():
    df_players.loc[index, 'whole_name'] = row['name']

## Tennis dataset

Remove duplicate rown in tennis dataset

In [ ]:
df_tennis.drop_duplicates(keep='first', inplace=True)

#### tourney_id

The tourney id that are null are replaced with the "unknown" value

In [ ]:
df_tennis['tourney_id'].isna().sum(axis = 0)
df_tennis.loc[df_tennis['tourney_id'].isna(), ['tourney_id']]='unknown'

#### winner_name and loser_name

For the winner and loser name the same changes of the male and female dataset are done to have the names with the same structure.

In [ ]:
#rows with nulls in name: replace null with unknown
df_tennis.loc[df_tennis['winner_name'].isna(), ['winner_name']]='unknown'
df_tennis['winner_name'].isna().sum(axis = 0)

df_tennis.loc[df_tennis['loser_name'].isna(), ['loser_name']]='unknown'
df_tennis['loser_name'].isna().sum(axis = 0)

In [ ]:
#nameswithout any letters are invalid: set to unknown
df_tennis.loc[~df_tennis['winner_name'].str.contains("[a-zA-Z]"), ['winner_name']]='unknown'
df_tennis.loc[~df_tennis['loser_name'].str.contains("[a-zA-Z]"), ['loser_name']]='unknown'

#replace numbers in names
df_tennis["winner_name"] = df_tennis["winner_name"].str.replace("[0-9]","",regex=True)
df_tennis["loser_name"] = df_tennis["loser_name"].str.replace("[0-9]","",regex=True)

#remove string between ()
for index, row in  df_tennis[df_tennis['winner_name'].str.contains('\(')].iterrows():
    pos = df_tennis.at[index,'winner_name'].find("(")
    df_tennis.at[index,'winner_name']= df_tennis.at[index,'winner_name'][0:pos]
for index, row in  df_tennis[df_tennis['loser_name'].str.contains('\(')].iterrows():
    pos = df_tennis.at[index,'loser_name'].find("(")
    df_tennis.at[index,'loser_name']= df_tennis.at[index,'loser_name'][0:pos]
    
#characters not indicated in the expression are considered invalid    
df_tennis["winner_name"] = df_tennis["winner_name"].str.replace("[^a-zA-Z ',-_.]","",regex=True)    
df_tennis["loser_name"] = df_tennis["loser_name"].str.replace("[^a-zA-Z ',-._]","",regex=True)

#replace allowed 
df_tennis["winner_name"] = df_tennis["winner_name"].str.replace("[^a-zA-Z ]"," ",regex=True)
df_tennis["loser_name"] = df_tennis["loser_name"].str.replace("[^a-zA-Z ]"," ",regex=True)

#trim apply
df_tennis["winner_name"] = df_tennis["winner_name"].str.strip()
df_tennis["loser_name"] = df_tennis["loser_name"].str.strip()
#df_tennis[['winner_name','loser_name']]

Remove duplicates intoduced by the changes

In [ ]:
#check if duplicates are been introduced
df_tennis.duplicated(keep='first').sum()
#df_tennis[df_tennis.duplicated(keep='first')==True]

In [ ]:
#remove duplicates
df_tennis.drop_duplicates(keep='first', inplace=True)

Remove rows where both winner and loser are unknown because the information about the players are not useful

In [ ]:
#remove (if there is) the row unknown-unknown for winner and (&) loser
row = df_tennis.index[(df_tennis['winner_name'].str.lower()=='unknown') 
                      & (df_tennis['loser_name'].str.lower()=='unknown')
                      & (df_tennis['winner_id'].isnull()) 
                      & (df_tennis['loser_id'].isnull()) 
                     ].tolist()
df_tennis.drop(row, inplace=True)

In [ ]:
#remove multiple " "
df_tennis['winner_name'] = df_tennis['winner_name'].map(lambda x: re.sub(' +', ' ', x))
df_tennis['loser_name'] = df_tennis['loser_name'].map(lambda x: re.sub(' +', ' ', x))

**Fixing names with more the one id binded**

In this section are handled the players that have linked to multiple ID. We have verified that these players have a maximum of two associated IDs so the reassignment is done using the ID that is most recurrent for that player.

In [ ]:
#Removing the multiple ids for one name WINNER
mul_ids = get_w_name_ids(df_tennis)
df_mul_ids = df_tennis[df_tennis['winner_name'].isin(mul_ids) & (df_tennis['winner_name'] != 'unknown')][['winner_name','winner_id']].sort_values(by=['winner_name','winner_id'])
w_names = df_mul_ids['winner_name'].unique().tolist()
w_ids = df_mul_ids['winner_id'].unique().tolist()
correct_w_id = {}
ids = df_mul_ids.groupby(['winner_name','winner_id'])['winner_id'].count().to_list()
i = 0
for name in w_names:
    if(ids[i] >= ids[i+1]):
        correct_w_id[name] = w_ids[i]
        df_tennis.loc[df_tennis['winner_name'] == name, 'winner_id'] = w_ids[i]
    else:
        correct_w_id[name] = w_ids[i+1]
        df_tennis.loc[df_tennis['winner_name'] == name, 'winner_id'] = w_ids[i+1]
    i+=2

In [ ]:
df_tennis.loc[df_tennis['loser_name'] == 'unknown', 'loser_name'] = None

In [ ]:
#Removing the multiple ids for one name LOSER
mul_ids = get_l_name_ids(df_tennis)
df_mul_ids = df_tennis[df_tennis['loser_name'].isin(mul_ids)][['loser_name','loser_id']].sort_values(by=['loser_name','loser_id'])
l_names = df_mul_ids['loser_name'].unique().tolist()
l_ids = df_mul_ids['loser_id'].unique().tolist()
correct_l_id = {}
ids = df_mul_ids.groupby(['loser_name','loser_id'])['loser_id'].count().to_list()
i = 0
for name in l_names:
    if(ids[i] >= ids[i+1]):
        correct_l_id[name] = l_ids[i]
        df_tennis.loc[df_tennis['loser_name'] == name, 'loser_id'] = l_ids[i]
    else:
        correct_l_id[name] = l_ids[i+1]
        df_tennis.loc[df_tennis['loser_name'] == name, 'loser_id'] = l_ids[i+1]
    i+=2

**Removing id binded with multiple names**

Given the presence of name pairs whose IDs are in common, it was necessary to reassign the IDs. To do this we first counted the <player, id> occurrences for each of the players, based on the count the ID was assigned to the player with the most occurrences. For the other player we first checked if there were other instances of him with different ID, otherwise he was assigned an incremental ID starting from the maximum ID present.

In [ ]:
# df_mul_names = df_tennis[df_tennis['winner_id'].isin(get_w_id_names())][['winner_name','winner_id']].sort_values(by=['winner_name','winner_id'])
# df_mul_names = df_mul_names.value_counts().reset_index()
# df_mul_names.columns = ['winner_name', 'winner_id', 'count']
# df_mul_names.sort_values(by='winner_id')

In [ ]:
df_mul_names = df_tennis[df_tennis['winner_id'].isin(get_w_id_names(df_tennis))][['winner_name','winner_id']].sort_values(by=['winner_name','winner_id'])
df_mul_names = df_mul_names.value_counts().reset_index()
df_mul_names.columns = ['winner_name', 'winner_id', 'count']
df_mul_names.sort_values(by='winner_id')
id_list =  df_mul_names['winner_id'].unique()
for w_id in id_list:
    p1 = df_mul_names[df_mul_names['winner_id'] == w_id]['winner_name'].to_list()[0]
    c1 = df_mul_names[df_mul_names['winner_id'] == w_id]['count'].to_list()[0]
    p2 = df_mul_names[df_mul_names['winner_id'] == w_id]['winner_name'].to_list()[1]
    c2 = df_mul_names[df_mul_names['winner_id'] == w_id]['count'].to_list()[1]
    new_id = max(df_tennis['winner_id'].max(),df_tennis['loser_id'].max()) + 1
    if(c1 >= c2):
        df_tennis.loc[df_tennis['winner_name'] == p2, 'winner_id'] = new_id
        df_tennis.loc[df_tennis['loser_name'] == p2, 'loser_id'] = new_id
    else:
        df_tennis.loc[df_tennis['winner_name'] == p1, 'winner_id'] = new_id
        df_tennis.loc[df_tennis['loser_name'] == p1, 'loser_id'] = new_id        

In [ ]:
df_mul_names = df_tennis[df_tennis['loser_id'].isin(get_l_id_names(df_tennis)) & (df_tennis['loser_name'] != 'unknown')][['loser_name','loser_id']].sort_values(by=['loser_name','loser_id'])
df_mul_names = df_mul_names.value_counts().reset_index()
df_mul_names.columns = ['loser_name', 'loser_id', 'count']
df_mul_names.sort_values(by='loser_id')
id_list =  df_mul_names['loser_id'].unique()
for l_id in id_list:
    p1 = df_mul_names[df_mul_names['loser_id'] == l_id]['loser_name'].to_list()[0]
    c1 = df_mul_names[df_mul_names['loser_id'] == l_id]['count'].to_list()[0]
    p2 = df_mul_names[df_mul_names['loser_id'] == l_id]['loser_name'].to_list()[1]
    c2 = df_mul_names[df_mul_names['loser_id'] == l_id]['count'].to_list()[1]
    new_id = max(df_tennis['winner_id'].max(),df_tennis['loser_id'].max()) + 1
    if(c1 >= c2):
        df_tennis.loc[df_tennis['winner_name'] == p2, 'winner_id'] = new_id
        df_tennis.loc[df_tennis['loser_name'] == p2, 'loser_id'] = new_id
    else:
        df_tennis.loc[df_tennis['winner_name'] == p1, 'winner_id'] = new_id
        df_tennis.loc[df_tennis['loser_name'] == p1, 'loser_id'] = new_id    

In [ ]:
df_tennis.loc[df_tennis['loser_name'].isna(), 'loser_name'] = 'unknown'

#### Merge names of datasets

Merge of names in tennis dataset and male + female dataset removing names that are not in tennis dataset. In this way we obtain a list of players in the tennis dataset with sex associated to them (that can be unknown)

In [ ]:
tennis_names = df_tennis['winner_name'].drop_duplicates().tolist()+ \
                df_tennis['loser_name'].drop_duplicates().tolist()
tennis_names = list(dict.fromkeys(tennis_names))

In [ ]:
df_names = pd.DataFrame(tennis_names, columns=['whole_name'])
df_names #10103 rows

In [ ]:
df_players

In [ ]:
df_players = df_names.merge(df_players, how='left', on='whole_name')
df_players

In [ ]:
#df_players[df_players['winner_name']=='Carlos  Cuevas']
df_players.loc[df_players['sex'].isna(),'sex']='U'

In [ ]:
#remove unknown 
row = df_players.index[df_players['whole_name'].str.lower()=='unknown'].tolist()
df_players.drop(row, inplace=True)

In [ ]:
df_players = df_players.drop(columns=['name', 'surname'])

In [ ]:
df_players

In [ ]:
df_tennis['winner_id'].max()

**Adding id column to player dataframe**

After having fixed the relationships between player and ID in the match dataset this information has been transferred in the table of players associating to each tennis player his unique ID.

In [ ]:
df_w = df_tennis[['winner_name','winner_id']].drop_duplicates()
df_l = df_tennis[['loser_name','loser_id']].drop_duplicates()
df_l = df_l.rename(columns={"loser_name": "winner_name", "loser_id": "winner_id"})
df = df_w.append(df_l)
df = df.set_index('winner_name')
df = df.dropna()

dict_name_id = df.to_dict()
dict_name_id = dict_name_id['winner_id']
ids = []
for name in df_players['whole_name']:
    if name in dict_name_id:
        ids.append(dict_name_id[name])
print(len(ids))
df_players['id'] = ids

In [ ]:
df_players

In [ ]:
df_tmp_names = df_players[['whole_name','id']]
df_tmp_names = df_tmp_names.set_index('whole_name')
df_tmp_ids = df_players[['whole_name','id']]
df_tmp_ids = df_tmp_ids.set_index('id')
dict_names_ids = df_tmp_names['id'].to_dict()
dict_ids_names = df_tmp_ids['whole_name'].to_dict()

In [ ]:
#fill missing winner_id or winner_id values
conditions = (df_tennis['winner_id'].isnull() ^ (df_tennis['winner_name'] == 'unknown'))
df_tmp = df_tennis[conditions][['winner_name','winner_id']]
for index, row in df_tmp.iterrows():
    if(row['winner_name'] == 'unknown'):
         if(row['winner_id'] in dict_ids_names.keys()):
                print(row['winner_name'], row['winner_id'])
                df_tennis.loc[df_tennis['winner_id'] == row['winner_id'], 'winner_name'] = dict_ids_names[row['winner_id']]
    else:
         if(row['winner_name'] in dict_names_ids.keys()):
            df_tennis.loc[df_tennis['winner_name'] == row['winner_name'], 'winner_id'] = dict_names_ids[row['winner_name']]

In [ ]:
#fill missing loser_id or loser_id values
conditions = (df_tennis['loser_id'].isnull() ^ (df_tennis['loser_name'] == 'unknown'))
df_tmp = df_tennis[conditions][['loser_name','loser_id']]
for index, row in df_tmp.iterrows():
    if(row['loser_name'] == 'unknown'):
         if(row['loser_id'] in dict_ids_names.keys()):
                df_tennis.loc[df_tennis['loser_id'] == row['loser_id'], 'loser_name'] = dict_ids_names[row['loser_id']]
    else:
         if(row['loser_name'] in dict_names_ids.keys()):
            df_tennis.loc[df_tennis['loser_name'] == row['loser_name'], 'loser_id'] = dict_names_ids[row['loser_name']]

In [ ]:
df_tennis.loc[df_tennis['winner_id'].isna(), 'winner_id'] = -1

In [ ]:
df_tennis.loc[df_tennis['loser_id'].isna(), 'loser_id'] = -1

#### Establish sex looking matches

Here sex for the players with sex unknown or incorrect (in matches male plays against male and female against female) is established looking at players of matches

In [ ]:
#establish sex for unknown and correct for known if there are errors
sex_dict = dict(zip(df_players.whole_name, df_players.sex))
dict_winner_match = df_tennis.groupby('winner_name')['loser_name'].unique().dropna().apply(list).to_dict()
dict_loser_match = df_tennis.groupby('loser_name')['winner_name'].unique().dropna().apply(list).to_dict()

In [ ]:
dict_winner_match

In [ ]:
#check if errors with F or M
error, erorr_rows_indexes = errors_match_sexs(sex_dict, dict_winner_match, dict_loser_match)

In [ ]:
erorr_rows_indexes

So, in each match male is against male and female against female (or unknown). Cases where this is not true because in some matches there are male and female mixerd, looking at these matches and there number we choose a sex rather than another for a player.

In [ ]:
changes = True
sex_dict = dict(zip(df_players.whole_name, df_players.sex))

while(changes):
    changes = False
    for key in sex_dict.keys():
        #consider the players as winner
        if sex_dict[key]=='U' and key in dict_winner_match.keys():
            losers = dict_winner_match[key]
            for loser in losers:
                if sex_dict[loser]!='U' :                    
                    df_players.loc[df_players['whole_name']==key, 'sex'] = sex_dict[loser]
                    changes = True
                    sex_dict[key] = sex_dict[loser]
                    break
            
        #consider the players as loser
        if sex_dict[key]=='U' and key in dict_loser_match.keys():
            winners = dict_loser_match[key]
            for winner in winners:
                if sex_dict[winner]!='U':
                    df_players.loc[df_players['whole_name']==key, 'sex'] = sex_dict[winner]
                    changes = True
                    sex_dict[key] = sex_dict[winner]
                    break

In [ ]:
#check if errors with F or M
error, erorr_rows_indexes = errors_match_sexs(sex_dict, dict_winner_match, dict_loser_match)

In [ ]:
erorr_rows_indexes = list(dict.fromkeys(erorr_rows_indexes))
erorr_rows_indexes

In [ ]:
for name_error in erorr_rows_indexes:
    sex = df_players[df_players['whole_name']==name_error]['sex'].tolist()[0]
    
    enemies = df_tennis[df_tennis['winner_name']==name_error]['loser_name'].tolist()
    enemies = enemies + df_tennis[df_tennis['loser_name']==name_error]['winner_name'].tolist()
    enemies = list(dict.fromkeys(enemies))
    
    female = 0
    male = 0
    
    for enemy in enemies:
        if sex_dict[enemy]=='F':
            female +=1
        else:
            male+=1
            
    if female>male:
        if sex_dict[name_error]!='F':
            print(error)
    else:
        if sex_dict[name_error]!='M':
            print(error)

In [ ]:
df_players[df_players['sex']=='U'].shape[0]

After the correction of sex, all the players have a known sex (female or male)

In [ ]:
df_players

#### tourney_name

Null values in the toruney name are replaced with an "unknown" value. For not null names, the validity of the names are analyzed cheching if the name has only alphabetic characters and some well defined special characters ("-","\_"). As was done for name of players, in this way we try to obtain the same tourney name for the same tourney removing characters invalid or looking for the same name written with small differences as, for example, the lower case instead the upper case.

In [ ]:
#rows with nulls in name: replace null with unknown
df_tennis.loc[df_tennis['tourney_name'].isna(), ['tourney_name']]='unknown'

In [ ]:
#names without any letters are invalid: set to unknown
df_tennis.loc[~df_tennis['tourney_name'].str.contains("[a-zA-Z]"), ['tourney_name']]='unknown'

#replace numbers in names
df_tennis["tourney_name"] = df_tennis["tourney_name"].str.replace("[0-9]","",regex=True)

#remove string between ()
for index, row in  df_tennis[df_tennis['tourney_name'].str.contains('\(')].iterrows():
    pos = df_tennis.at[index,'tourney_name'].find("(")
    df_tennis.at[index,'tourney_name']= df_tennis.at[index,'tourney_name'][0:pos]
    
#characters not indicated in the expression are considered invalid    
df_tennis["tourney_name"] = df_tennis["tourney_name"].str.replace("[^a-zA-Z -_]","",regex=True)
df_tennis["tourney_name"] = df_tennis["tourney_name"].str.replace("[^a-zA-Z ]"," ",regex=True)

#trim apply
df_tennis["tourney_name"] = df_tennis["tourney_name"].str.strip()
df_tennis

Then, we try to remove duplicate rows in the dataset introduced with the changes

In [ ]:
#check if duplicates are been introduced
df_tennis.duplicated(keep='first').sum()

In [ ]:
#remove duplicates
df_tennis.drop_duplicates(keep='first', inplace=True)
#rows with unknown tourney_name: 21. All of these have null tourney_id

In [ ]:
#remove multiple " "
df_tennis['tourney_name'] = df_tennis['tourney_name'].map(lambda x: re.sub(' +', ' ', x))

After the correction there are still some tourney with more names but these are all similar. For example are of the form: \['Indian Wells', 'Indian Wells Masters'\] or \['US Open', 'Us Open'\]. So, we try to have all lower to resolve problem of upper case and in cases like the first we took the longer name ('Indian Wells Masters')

In [ ]:
#make the name equal for similar names (es: Madrid, Madrid Master associated to same tourney_id)
dict_tourney_id = df_tennis.groupby('tourney_id')['tourney_name'].unique().apply(list).to_dict()

In [ ]:
#print tourneys with more names
keys = []
for key, value in dict_tourney_id.items():
    if len(value)>1:
        print(key, value)
        keys.append(keys)

In [ ]:
for key, value in dict_tourney_id.items():
    if len(value)==2:
        if value[0].lower() == value[1].lower():
            df_tennis.loc[df_tennis['tourney_id']==key, ['tourney_name']]=value[0]
        elif value[0]=='unknown':
            df_tennis.loc[df_tennis['tourney_id']==key, ['tourney_name']]=value[1]
        elif value[1]=='unknown':
            df_tennis.loc[df_tennis['tourney_id']==key, ['tourney_name']]=value[0]
        elif value[0].lower() in value[1].lower():
            df_tennis.loc[df_tennis['tourney_id']==key, ['tourney_name']]=value[1]
        elif value[1].lower() in value[0].lower():
            df_tennis.loc[df_tennis['tourney_id']==key, ['tourney_name']]=value[0]

#### winner_hand and loser_hand

Here we establish what is the hand used by the players to play. To do this we analyze the matches played by every player. We noticed that every player always play with the same hand, so, if in at least one match of a player there is a left or right hand, we associate this to the player. For the other players the hand is considered unknown ("U")

In [ ]:
df_players['hand']='U'

In [ ]:
#substitute nulls in hand in tennis with 'U'
df_tennis.loc[df_tennis['winner_hand'].isna(), ['winner_hand']]='U'
df_tennis.loc[df_tennis['loser_hand'].isna(), ['loser_hand']]='U'

In [ ]:
df_tennis[(df_tennis['winner_hand']=='R') | (df_tennis['winner_hand']=='L')] #136653 rows

In [ ]:
winner_hand_dict = df_tennis.groupby(['winner_id','winner_name'])['winner_hand'].unique().apply(list).to_dict()
loser_hand_dict = df_tennis.groupby(['loser_id','loser_name'])['loser_hand'].unique().apply(list).to_dict()

In [ ]:
for key, value in winner_hand_dict.items():
    if key in loser_hand_dict.keys() and value[0] not in loser_hand_dict[key]:
        loser_hand_dict[key].append(value[0])
    else:
        loser_hand_dict[key] = value

In [ ]:
for key, value in loser_hand_dict.items():
    if 'U' in loser_hand_dict[key] and ('L' in loser_hand_dict[key] or 'R' in loser_hand_dict[key]):
        
        print(key)
        print(key[1])
        print(loser_hand_dict[key])
        
        if(loser_hand_dict[key][0]!='U'):
            df_players.loc[df_players['whole_name']==key[1],'hand']= loser_hand_dict[key][0]
            df_tennis.loc[df_tennis['winner_name']==key[1],'winner_hand']=loser_hand_dict[key][0]
            df_tennis.loc[df_tennis['loser_name']==key[1], 'loser_hand']=loser_hand_dict[key][0]
        else:
            df_players.loc[df_players['whole_name']==key[1],'hand']= loser_hand_dict[key][1]
            df_tennis.loc[df_tennis['winner_name']==key[1], 'winner_hand']=loser_hand_dict[key][1]
            df_tennis.loc[df_tennis['loser_name']==key[1], 'loser_hand']=loser_hand_dict[key][1]
            
    elif 'L' in loser_hand_dict[key] or 'R' in loser_hand_dict[key]:
        df_players.loc[df_players['whole_name']==key[1],'hand']= loser_hand_dict[key][0]

In [ ]:
#correct hand in df_tennis for future possible analysis
df_tennis[df_tennis['winner_name']=='Ekaterina Makarova']['winner_hand'].unique()

In [ ]:
df_players[df_players['hand']!='U']

We estabilished the hand for 3804 players

#### rank and point_rank

For each player we define what is the best rank reached in the classification and the maximum number of points gained. To do this we analyze all the match of a player and we take respectively the minimum and the maximum value of the columns. If a player has all null values for these columns we use 0 as default value. We use a default value also to correct null values in the tennis dataset

In [ ]:
df_players['best_rank'] = 0 #default value for unknown
df_players['best_rank_points'] = 0 #default value for unknown

In [ ]:
dict_winner_rank = df_tennis[~df_tennis['winner_rank'].isna()].groupby('winner_name', dropna=True)['winner_rank'].unique().apply(list).to_dict()
dict_loser_rank = df_tennis[~df_tennis['loser_rank'].isna()].groupby('loser_name', dropna=True)['loser_rank'].unique().apply(list).to_dict()

dict_winner_rank_points = df_tennis[~df_tennis['winner_rank_points'].isna()].groupby('winner_name', dropna=True)['winner_rank_points'].unique().apply(list).to_dict()
dict_loser_rank_points = df_tennis[~df_tennis['loser_rank_points'].isna()].groupby('loser_name', dropna=True)['loser_rank_points'].unique().apply(list).to_dict()

In [ ]:
for index, row in  df_players.iterrows():  
    
    #rank
    ranks = []
    if row['whole_name'] in dict_winner_rank:
        ranks = ranks + dict_winner_rank[row['whole_name']]
    if row['whole_name'] in dict_loser_rank:
        ranks = ranks + dict_loser_rank[row['whole_name']]
     
    ranks = list(dict.fromkeys(ranks))
    if len(ranks)>0:
        df_players.at[index, 'best_rank'] = min(ranks)
    
    #points
    ranks_point = []
    if row['whole_name'] in dict_winner_rank_points:
        ranks_point = ranks + dict_winner_rank_points[row['whole_name']]
    if row['whole_name'] in dict_loser_rank_points:
        ranks_point = ranks + dict_loser_rank_points[row['whole_name']]
    
    ranks_point = list(dict.fromkeys(ranks_point))
    if len(ranks_point)>0:
        df_players.at[index, 'best_rank_points'] = max(ranks_point)
    
df_tennis

In [ ]:
#df_players[df_players['best_rank']==1]
df_players

In [ ]:
df_tennis.loc[df_tennis['winner_rank'].isna(), 'winner_rank'] = 0
df_tennis.loc[df_tennis['winner_rank_points'].isna(), 'winner_rank_points'] = 0

df_tennis.loc[df_tennis['loser_rank'].isna(), 'loser_rank'] = 0
df_tennis.loc[df_tennis['loser_rank_points'].isna(), 'loser_rank_points'] = 0

#### best of and score

Here we correct the best of value looking at the score and the tennis rules. If analyzing a score, a score is invalid for a best of 3 matches, but it is valid for best of 5 matches, we correct the 3 with 5. The same reasoning is done 

In [ ]:
#change best of 3 in 5 if matches are valid for best of 5 (best_of not null)
for index, row in  df_tennis[df_tennis['best_of']==3].iterrows():
    if isinstance(row['score'], str):
        sets = row['score'].split( )

        if not validity_match(sets, 3) and validity_match(sets, 5):
            df_tennis.loc[index, 'best_of'] = 5
        elif validity_match(sets, 5) and not is_best_of_3(sets)and is_best_of_5(sets):
            df_tennis.loc[index, 'best_of'] = 5
        

#change best of 5 in 3 if matches are valid for best of 3
for index, row in  df_tennis[df_tennis['best_of']==5].iterrows():
    if isinstance(row['score'], str):
        sets = row['score'].split( )

        if not validity_match(sets, 5) and validity_match(sets, 3):
            df_tennis.loc[index, 'best_of'] = 3
        elif validity_match(sets, 3) and is_best_of_3(sets) and not is_best_of_5(sets):
            df_tennis.loc[index, 'best_of'] = 3

In [ ]:
df_tennis[df_tennis['best_of'].isna()]['score'].dropna()

In [ ]:
for index, row in  df_tennis[df_tennis['best_of'].isna()].iterrows():
    if isinstance(row['score'], str):
        sets = row['score'].split( )

        if validity_match(sets, 3) and not validity_match(sets, 5):
            df_tennis.loc[index, 'best_of'] = 3
        elif not validity_match(sets, 3) and validity_match(sets, 5):
            df_tennis.loc[index, 'best_of'] = 5
        elif validity_match(sets, 3) and validity_match(sets, 5):
            if is_best_of_3(sets) and not is_best_of_5(sets):
                df_tennis.loc[index, 'best_of'] = 3
            elif not is_best_of_3(sets) and is_best_of_5(sets):
                df_tennis.loc[index, 'best_of'] = 5

In [ ]:
for index, row in df_tennis[df_tennis['best_of'].isna()][['tourney_id','tourney_name']].drop_duplicates().iterrows():
    best_ofs = df_tennis[(df_tennis['tourney_id']==row['tourney_id']) & (df_tennis['tourney_name']==row['tourney_name'])]['best_of'].dropna().tolist()
    values = list(dict.fromkeys(best_ofs))
    if len(values) == 1:
        df_tennis.loc[(df_tennis['tourney_id']==row['tourney_id']) & (df_tennis['tourney_name']==row['tourney_name']), 'best_of'] = values[0]

If a score is invalid following the tennis rules the value is corrected with an empty value (""). We have found 5155 invalid scores

In [ ]:
df_tennis.loc[df_tennis['score'].isna(), 'score'] = ""

In [ ]:
count = 0
for index, row in  df_tennis.iterrows():
    if row['score']!="":
        sets = row['score'].split()
        if not Walkover(sets[0]) and not Bye(sets[0]) and not Retirement(sets[0]) and not Default(sets[0]) \
            and not validity_match(sets, 3) and not validity_match(sets, 5):
            df_tennis.loc[index, 'score'] = ""
            count +=1
        
count

In [ ]:
#wrong scores
count

#### num of best of 3/5 played and wins

For each player we count the number of matches best of 3 and best of 5 played and won counting the number of matches.

In [ ]:
df_players['best_of_3_match'] = 0
df_players['best_of_5_match'] = 0
df_players['best_of_3_wins'] = 0
df_players['best_of_5_wins'] = 0

In [ ]:
winners_best_3 = df_tennis[(df_tennis['winner_name']!=df_tennis['loser_name']) & (df_tennis['best_of']==3)]['winner_name'].tolist()
losers_best_3 = df_tennis[(df_tennis['winner_name']!=df_tennis['loser_name']) & (df_tennis['best_of']==3)]['loser_name'].tolist()

In [ ]:
winners_best_5 = df_tennis[(df_tennis['winner_name']!=df_tennis['loser_name']) & (df_tennis['best_of']==5)]['winner_name'].tolist()
losers_best_5 = df_tennis[(df_tennis['winner_name']!=df_tennis['loser_name']) & (df_tennis['best_of']==5)]['loser_name'].tolist()

In [ ]:
for index, row in  df_players.iterrows(): 
    num_w = winners_best_3.count(row['whole_name'])
    num_l = losers_best_3.count(row['whole_name'])
    
    df_players.at[index, 'best_of_3_match'] = num_w + num_l
    if num_w!=0:
        df_players.at[index, 'best_of_3_wins'] = num_w
        
    num_w = winners_best_5.count(row['whole_name'])
    num_l = losers_best_5.count(row['whole_name'])
    
    df_players.at[index, 'best_of_5_match'] = num_w + num_l
    if num_w!=0:
        df_players.at[index, 'best_of_5_wins'] = num_w

In [ ]:
df_players

## IOC

The International Olympic Committee (IOC) uses three-letter abbreviation country codes to refer to each group of athletes participating in the Olympic Games. Each code usually identifies a National Olympic Committee.

Search the dataset for the players' iocs to insert them where they are missing

In [ ]:
df_players.insert(2, 'ioc', None)

In [ ]:
for index, row in df_players.iterrows():
    ioc = None
    if df_tennis[df_tennis['winner_id'] == row['id']][['winner_ioc']].shape[0] > 0:
        ioc = df_tennis[df_tennis['winner_id'] == row['id']][['winner_ioc']].iloc[0]
    if ioc is None:    
        if df_tennis[df_tennis['loser_id'] == row['id']][['loser_ioc']].shape[0] > 0:
            ioc = df_tennis[df_tennis['loser_id'] == row['id']][['loser_ioc']].iloc[0]
    df_players.loc[df_players['id']==row['id'], 'ioc']=ioc.iloc[0]
    # for speedup debug uncomment...
    #if index == 100:
    #    break
df_players[~df_players['ioc'].isnull()]

#### surface
It inserts the surface where it is missing, it takes the surface from other matches of the same tournament. <br />
Where it fails it puts unknown

In [ ]:
# prepare a dictionary of tourney_name and surface where there are no null
df_surface = df_tennis[['tourney_name','surface']].drop_duplicates().dropna()
surface_dict = df_surface.set_index('tourney_name').to_dict()['surface']

# set the null value with the value found for the same tournament
null_surface = df_tennis.surface.isna()
df_tennis.loc[null_surface, 'surface'] = df_tennis.tourney_name.map(surface_dict)


# set the remaining surface null to unknown
df_tennis.loc[df_tennis['surface'].isna(), ['surface']]='unknown'

# print the result
df_tennis['surface'].value_counts().sort_index()

#### draw_size
It inserts the draw_size where it is missing, it takes the draw_size from other matches of the same tournament. <br />
Where it fails it puts -1

In [ ]:
# prepare a dictionary of tourney_name and draw_size where there are no null
df_draw_size = df_tennis[['tourney_name','draw_size']].drop_duplicates().dropna()
draw_size_dict = df_draw_size.set_index('tourney_name').to_dict()['draw_size']

# set the null value with the value found for the same tournament
null_draw_size = df_tennis.draw_size.isna()
df_tennis.loc[null_draw_size, 'draw_size'] = df_tennis.tourney_name.map(draw_size_dict)


# set the remaining draw_size null to -1
df_tennis.loc[df_tennis['draw_size'].isna(), ['draw_size']] = -1

# print the result
df_tennis['draw_size'].value_counts().sort_index()

#### tourney_level
It inserts the tourney_level where it is missing, it takes the tourney_level from other matches of the same tournament. <br />
Where it fails it puts unknown

In [ ]:
# prepare a dictionary of tourney_name and tourney_level where there are no null
df_tourney_level = df_tennis[['tourney_name','tourney_level']].drop_duplicates().dropna()
tourney_level_dict = df_tourney_level.set_index('tourney_name').to_dict()['tourney_level']

# set the null value with the value found for the same tournament
null_tourney_level = df_tennis.tourney_level.isna()
df_tennis.loc[null_tourney_level, 'tourney_level'] = df_tennis.tourney_name.map(tourney_level_dict)


# set the remaining tourney_level null to unknown
df_tennis.loc[df_tennis['tourney_level'].isna(), ['tourney_level']] = 'unknown'

# print the result
df_tennis['tourney_level'].value_counts().sort_index()

#### match_num
In this part, we extract statistics on a match played and winned for each year and the null values are replaced with -1.

In [ ]:
# set the null value to -1
df_tennis.loc[df_tennis['match_num'].isna(), 'match_num'] = -1
df_tennis[df_tennis['match_num'].isnull()]

In [ ]:
df_players['m_16'] = -1
df_players['w_16'] = -1
df_players['m_17'] = -1
df_players['w_17'] = -1
df_players['m_18'] = -1
df_players['w_18'] = -1
df_players['m_19'] = -1
df_players['w_19'] = -1
df_players['m_20'] = -1
df_players['w_20'] = -1
df_players['m_21'] = -1
df_players['w_21'] = -1

In [ ]:
years = ['2016', '2017', '2018', '2019', '2020', '2021']
df_tmp = df_tennis[['tourney_id', 'winner_id', 'loser_id']]
year = df_tmp['tourney_id'].map(lambda x: x[0:4] if x[0:4] in years else x)
df_tmp['year'] = year
df_tmp.drop(columns = 'tourney_id')

w_count = df_tmp.groupby(by = ['winner_id', 'year'])['tourney_id'].count().reset_index(name="count")
l_count = df_tmp.groupby(by = ['loser_id', 'year'])['tourney_id'].count().reset_index(name="count")

for player in df_players['id']:
    for year in years:
        col_name_m = 'm_' + year[2:4]
        col_name_w = 'w_' + year[2:4]
        winned_match = w_count[(w_count['winner_id'] == player) & (w_count['year'] == year)]['count']
        losed_match = l_count[(l_count['loser_id'] == player) & (l_count['year'] == year)]['count']
        if (len(winned_match)):
            winned_match = int(winned_match)
            df_players.loc[df_players['id'] == player, col_name_w] = winned_match
        else:
            winned_match = 0
        if (len(losed_match)):
            losed_match = int(losed_match)
            df_players.loc[df_players['id'] == player, col_name_m] = winned_match + losed_match

In [ ]:
df_players

#### tourney_spectators
It inserts the tourney_spectators where it is missing, it takes the tourney_spectators from other matches of the same tournament. <br />
Where it fails it puts -1

In [ ]:
# prepare a dictionary of tourney_name and tourney_spectators where there are no null
df_tourney_spectators = df_tennis[['tourney_name','tourney_spectators']].drop_duplicates().dropna()
tourney_spectators_dict = df_tourney_spectators.set_index('tourney_name').to_dict()['tourney_spectators']

# set the null value with the value found for the same tournament
null_tourney_spectators = df_tennis.tourney_spectators.isna()
df_tennis.loc[null_tourney_spectators, 'tourney_spectators'] = df_tennis.tourney_name.map(tourney_spectators_dict)

df_tennis.loc[df_tennis['tourney_spectators'].isna(), ['tourney_spectators']] = -1


df_tennis[df_tennis['tourney_spectators'].isnull()]

#### tourney_date
Set date in last tournament Taipei, the only tournement where date is missing.

In [ ]:
# The only tournament that has some matches with null date is the last one in Taipei in the dataframe. 
# Let's put the date 20171113.0 that corresponds to the matches of that tournament
df_tennis.loc[(df_tennis['tourney_name'] == 'Taipei') & (df_tennis['tourney_date'].isnull()), 'tourney_date'] = '20171113.0'
df_tennis[df_tennis['tourney_date'].isnull()]

#### round

Using the column *round* has been possible to understand how much tourney the player has won. Looking for the rows in which his\she is the winner and round has value 'F'.

In [ ]:
# set the null value to u (unkown)
df_tennis.loc[df_tennis['round'].isna(), 'round'] = 'U'
df_tennis[df_tennis['round'].isnull()]

In [ ]:
df_players['w_tourney'] = 0
winned_finals = df_tennis[(~df_tennis['round'].isnull()) & (df_tennis['round'] == 'F')].groupby('winner_name')['round'].agg('count').to_dict()
for player in winned_finals.keys():
    df_players.loc[df_players['whole_name'] == player, 'w_tourney'] = winned_finals[player]

In [ ]:
df_players

#### tourney_revenue
It inserts the tourney_revenue where it is missing, it takes the tourney_revenue from other matches of the same tournament. <br />
Where it fails it puts unknown

In [ ]:
# prepare a dictionary of tourney_name and tourney_revenue where there are no null
df_tourney_revenue = df_tennis[['tourney_name','tourney_revenue']].drop_duplicates().dropna()
tourney_revenue_dict = df_tourney_revenue.set_index('tourney_name').to_dict()['tourney_revenue']

# set the null value with the value found for the same tournament
null_tourney_revenue = df_tennis.tourney_revenue.isna()
df_tennis.loc[null_tourney_revenue, 'tourney_revenue'] = df_tennis.tourney_name.map(tourney_revenue_dict)


df_tennis[df_tennis['tourney_revenue'].isnull()]

#### minutes

We try to remove outliers in the minutes of matches and to do this we used the threashold given by longest matches in the tennis history. To correct outliers and nulls, if the score is a valid score, we have estimate a the duration of the match calculating an average number of minutes for the games and looking at the number of games played in the matche we calculate the minutes. If the score is not valid we have used the mean of minutes for best of 3 or best of 5 matches looking at the best of value.

In [ ]:
#use of current world records of time
df_tennis.loc[(df_tennis['minutes'] > 391) & (df_tennis['best_of']==3), 'minutes'] = np.nan 
df_tennis.loc[(df_tennis['minutes'] > 665) & (df_tennis['best_of']==5), 'minutes'] = np.nan 

In [ ]:
avg_time_best_of_3 = df_tennis[df_tennis['best_of']==3]['minutes'].mean() #mean after correction of errors
avg_time_best_of_5 = df_tennis[df_tennis['best_of']==5]['minutes'].mean() #mean after correction of errors

In [ ]:
#calculate the avg time for a game
number_games = 0
total_minutes = 0

for index, row in df_tennis[~df_tennis['minutes'].isna()].iterrows():
    if validity_match(row['score'].split(), row['best_of']):
        num = count_games(row['score'])
        if num >= 0:
            number_games += num
            total_minutes += row['minutes'] #not null because iterrows

In [ ]:
minutes_per_game = total_minutes/number_games
minutes_per_game

In [ ]:
for index, row in df_tennis[(df_tennis['minutes'].isna()) & (df_tennis['score']!="")].iterrows():
    sets = row['score'].split()
    if len(sets) == 0:
        continue
    elif len(sets)==1 and Retirement(sets[0]) or Default(sets[0]) or Bye(sets[0]) or Walkover(sets[0]):
        df_tennis.loc[index, 'minutes'] = 0
    elif len(sets)>=1 and validity_match(row['score'].split(), row['best_of']):
        df_tennis.loc[index, 'minutes'] = count_games(row['score']) * minutes_per_game

In [ ]:
for index, row in df_tennis[(df_tennis['minutes']==0) & (df_tennis['score']!="")].iterrows():
    if IsMatchWithZeroIncorrect(row['score'])==True:
        sets = row['score'].split()
        if len(sets) == 0:
            continue
        elif len(sets)==1 and Retirement(sets[0]) or Default(sets[0]) or Bye(sets[0]) or Walkover(sets[0]):
            df_tennis.loc[index, 'minutes'] = 0
        elif len(sets)>=1 and validity_match(row['score'].split(), row['best_of']):
            df_tennis.loc[index, 'minutes'] = count_games(row['score']) * minutes_per_game

In [ ]:
df_tennis.loc[(df_tennis['minutes'].isna()) & (df_tennis['best_of']==3), 'minutes'] = avg_time_best_of_3
df_tennis.loc[(df_tennis['minutes'].isna()) & (df_tennis['best_of']==5), 'minutes'] = avg_time_best_of_5

In [ ]:
df_tennis.loc[df_tennis['minutes'].isna()]

In the player dataset, we insert for each player the total minutes played (as winner or loser) summing the minutes of all their matches

In [ ]:
#statistics players: number of minutes xe played in total
df_players['tot_minutes'] = 0

In [ ]:
minutes_dict = df_tennis[df_tennis['minutes']>=0].groupby('winner_name')['minutes'].apply(list).to_dict()
for key in minutes_dict.keys():
    minutes_dict[key]= sum(minutes_dict[key])

In [ ]:
keys = minutes_dict.keys()

In [ ]:
loser_dict = df_tennis[df_tennis['minutes']>=0].groupby('loser_name')['minutes'].apply(list).to_dict()
for key in loser_dict.keys():
    loser_dict[key]= sum(loser_dict[key])
    if key in keys:
        minutes_dict[key] = minutes_dict[key] + loser_dict[key]
    else:
        minutes_dict[key] = loser_dict[key]

In [ ]:
for key in minutes_dict.keys():
    df_players.loc[df_players['whole_name']==key, 'tot_minutes'] = minutes_dict[key]

In [ ]:
df_players

#### winner_id

In [ ]:
df_tennis[df_tennis['winner_id'].isnull()]

#### loser_id

In [ ]:
df_tennis[df_tennis['loser_id'].isnull()]["loser_name"]
df_tennis['winner_entry'].unique()

#### winner_entry

In [ ]:
df_tennis[df_tennis['loser_entry'].isnull()]

In [ ]:
df_tennis['winner_entry'].fillna(value='U', inplace=True)

#### loser_entry

In [ ]:
df_tennis[df_tennis['loser_entry'].isnull()]

In [ ]:
df_tennis['loser_entry'].fillna(value='U', inplace=True)

#### winner_ioc

Search the dataset for the players' iocs to insert them where they are missing

In [ ]:
# I take the winner_ioc null, of these I take winner_id and winner_name, 
# and for each one I search in the dataset and I complete
df_winner_ioc = df_tennis[
    (df_tennis['winner_ioc'].isnull()) & 
    (
        (df_tennis['winner_name']!='unknown') | (df_tennis['winner_id'].notnull())
    )
][["winner_id", "winner_name"]].drop_duplicates()

# iterate over winner_ioc
for index, row in df_winner_ioc.iterrows():
    tmp_w_ioc = None
    
    if row['winner_id'] is not None:
        # if winner_id is not null use it
        tmp_w_ioc = df_tennis[(df_tennis['winner_id']==row['winner_id']) & (df_tennis['winner_ioc'].notnull())]['winner_ioc'].iloc[0]
        df_tennis.loc[df_tennis['winner_id']==row['winner_id'], ['winner_ioc']] = tmp_w_ioc        
    else:
        # if winner_id is null use winner_name
        tmp_w_ioc = df_tennis[(df_tennis['winner_name']==row['winner_name']) & (df_tennis['winner_ioc'].notnull())]['winner_ioc'].iloc[0]    
        df_tennis.loc[df_tennis['winner_name']==row['winner_name'], ['winner_ioc']] = tmp_w_ioc

df_tennis.loc[df_tennis['winner_ioc'].isnull(),'winner_ioc']='unknown'    
df_winner_ioc = df_tennis[((df_tennis['winner_ioc']=='unknown'))][["tourney_id","tourney_name", "winner_id", "winner_name", "winner_ioc", "loser_id", "loser_name", "loser_ioc"]]
df_winner_ioc

#### loser_ioc
Search the dataset for the players' iocs to insert them where they are missing

In [ ]:
# I take the loser_ioc null, of these I take loser_id and loser_name, 
# and for each one I search in the dataset and I complete
df_loser_ioc = df_tennis[
    (df_tennis['loser_ioc'].isnull()) & 
    (
        (df_tennis['loser_name']!='unknown') | (df_tennis['loser_id'].notnull())
    )
][["loser_id", "loser_name"]].drop_duplicates()

# iterate over loser_ioc
for index, row in df_loser_ioc.iterrows():
    tmp_l_ioc = None
    # if loser_id is not null use it
    if row['loser_id'] is not None:
        tmp_l_ioc = df_tennis[
            (df_tennis['loser_id']==row['loser_id']) & 
           (df_tennis['loser_ioc'].notnull())]['loser_ioc'].iloc[0]
        df_tennis.loc[df_tennis['loser_id']==row['loser_id'], ['loser_ioc']] = tmp_l_ioc
    # if loser_id is null use loser_name
    else:
        tmp_l_ioc = df_tennis[
                (df_tennis['loser_name']==row['loser_name']) & 
                (df_tennis['loser_ioc'].notnull())]['loser_ioc'].iloc[0]    
        df_tennis.loc[df_tennis['loser_name']==row['loser_name'], ['loser_ioc']] = tmp_l_ioc
    #print(tmp_l_ioc)

df_tennis.loc[df_tennis['loser_ioc'].isnull(),'loser_ioc']='unknown'    
df_loser_ioc = df_tennis[((df_tennis['loser_ioc']=='unknown'))][["tourney_id","tourney_name", "winner_id", "winner_name", "winner_ioc", "loser_id", "loser_name", "loser_ioc"]]
df_loser_ioc

#### winner_ht and loser_ht



In [ ]:
bp = pd.DataFrame.boxplot(df_tennis['winner_ht'], return_type='dict')

In [ ]:
outliers_ht = [flier.get_ydata() for flier in bp["fliers"]][0]
outliers_ht = list(dict.fromkeys(outliers_ht))

In [ ]:
w_names = df_tennis[df_tennis['winner_ht'].isin(outliers_ht)]['winner_name'].unique().tolist()

In [ ]:
l_names = df_tennis[df_tennis['loser_ht'].isin(outliers_ht)]['loser_name'].unique().tolist()

In [ ]:
w_names.sort() == l_names.sort() #True

In [ ]:
df_tennis.loc[df_tennis['winner_name'].isin(w_names), 'winner_ht'] = np.nan
df_tennis.loc[df_tennis['loser_name'].isin(w_names), 'loser_ht'] = np.nan

In [ ]:
male_names = df_players[df_players['sex']=='M']['whole_name'].tolist()
female_names = df_players[df_players['sex']=='F']['whole_name'].tolist()

In [ ]:
#corrections using sex and ioc

In [ ]:
df_ht = pd.concat([df_tennis[['winner_name', 'winner_ht', 'winner_ioc']].rename(columns={"winner_name": "name", "winner_ht": "ht", "winner_ioc": "ioc"}), df_tennis[['loser_name', 'loser_ht', 'loser_ioc']].rename(columns={"loser_name": "name", "loser_ht": "ht", "loser_ioc": "ioc"})]).drop_duplicates()

In [ ]:
df_ht

In [ ]:
male_avg_ht = df_ht[(df_ht['name'].isin(male_names))].groupby('ioc').mean().reset_index()

In [ ]:
for index, row in male_avg_ht[~male_avg_ht['ht'].isna()].iterrows():
    df_tennis.loc[(df_tennis['winner_ht'].isna()) & (df_tennis['winner_name'].isin(male_names)) & (df_tennis['winner_ioc']==row['ioc']), 'winner_ht']=row['ht']
    df_tennis.loc[(df_tennis['loser_ht'].isna()) & (df_tennis['loser_name'].isin(male_names)) & (df_tennis['loser_ioc']==row['ioc']), 'loser_ht']=row['ht']

In [ ]:
female_avg_ht = df_ht[(df_ht['name'].isin(female_names))].groupby('ioc').mean().reset_index()

In [ ]:
for index, row in female_avg_ht[~female_avg_ht['ht'].isna()].iterrows():
    df_tennis.loc[(df_tennis['winner_ht'].isna()) & (df_tennis['winner_name'].isin(female_names)) & (df_tennis['winner_ioc']==row['ioc']), 'winner_ht']=row['ht']
    df_tennis.loc[(df_tennis['loser_ht'].isna()) & (df_tennis['loser_name'].isin(female_names)) & (df_tennis['loser_ioc']==row['ioc']), 'loser_ht']=row['ht']

In [ ]:
generic_avg_ht = df_ht[(df_ht['name'].isin(female_names))].groupby('ioc').mean().reset_index()

In [ ]:
for index, row in generic_avg_ht[~generic_avg_ht['ht'].isna()].iterrows():
    df_tennis.loc[(df_tennis['winner_ht'].isna()) & (df_tennis['winner_ioc']==row['ioc']), 'winner_ht']=row['ht']
    df_tennis.loc[(df_tennis['loser_ht'].isna()) & (df_tennis['loser_ioc']==row['ioc']), 'loser_ht']=row['ht']

In [ ]:
#correction using sex

In [ ]:
male_mean = df_ht[df_ht['name'].isin(male_names)]['ht'].mean()

In [ ]:
female_mean = df_ht[df_ht['name'].isin(female_names)]['ht'].mean()

In [ ]:
df_tennis.loc[(df_tennis['winner_ht'].isna()) & (df_tennis['winner_name'].isin(male_names)), 'winner_ht']=male_mean
df_tennis.loc[(df_tennis['loser_ht'].isna()) & (df_tennis['loser_name'].isin(male_names)), 'loser_ht']=male_mean

In [ ]:
df_tennis.loc[(df_tennis['winner_ht'].isna()) & (df_tennis['winner_name'].isin(female_names)), 'winner_ht']=female_mean
df_tennis.loc[(df_tennis['loser_ht'].isna()) & (df_tennis['loser_name'].isin(female_names)), 'loser_ht']=female_mean

In [ ]:
df_players['height'] = -1

In [ ]:
df_tennis['loser_ht'].fillna(value=df_ht['ht'].mean(), inplace=True) 

In [ ]:
df_tennis['winner_ht'].fillna(value=df_ht['ht'].mean(), inplace=True) 

In [ ]:
df_tennis['winner_ht'] = df_tennis['winner_ht'].map(round_2)
df_tennis['loser_ht'] = df_tennis['loser_ht'].map(round_2)

In [ ]:
win_name_ht = dict(zip(df_tennis.winner_name, df_tennis.winner_ht))
loser_name_ht = dict(zip(df_tennis.loser_name, df_tennis.loser_ht))

names_ht_dict = win_name_ht.copy()
names_ht_dict.update(loser_name_ht)

In [ ]:
for index, row in df_players.iterrows():
    if row['whole_name'] in names_ht_dict:
        df_players.loc[index, 'height'] = names_ht_dict[row['whole_name']]

### AGE

Age is expressed in days from birth. Where it is missing is calculated using a set value, calculating the date of birth and making the difference with the date of the tournament and recalculating the days

In [ ]:
age_limit = 80
df_tennis.loc[df_tennis['winner_age'] > age_limit, 'winner_age'] = np.nan
df_tennis.loc[df_tennis['loser_age'] > age_limit, 'loser_age'] = np.nan

df_tennis_tmp = df_tennis.copy()

winner = df_tennis_tmp[df_tennis_tmp['winner_age'].isnull() & df_tennis_tmp['tourney_date'].notnull()]['winner_name'].drop_duplicates().tolist()
loser  = df_tennis_tmp[df_tennis_tmp['loser_age'].isnull() & df_tennis_tmp['tourney_date'].notnull()]['loser_name'].drop_duplicates().tolist()
merged = list(set(winner + loser))

for pl_name in merged:
    tourney_date = None
    pl_age = None
    
    ref = df_tennis_tmp[(df_tennis_tmp['winner_age'].notnull()) & (df_tennis_tmp['winner_name']==pl_name) & (df_tennis_tmp['tourney_date'].notnull())]
    if len(ref) != 0:
        pl_age = ref.iloc[0]['winner_age']
        tourney_date = ref.iloc[0]['tourney_date']
    elif len(ref) != 0:
        ref = df_tennis_tmp[(df_tennis_tmp['loser_age'].notnull()) & (df_tennis_tmp['loser_name']==pl_name) & (df_tennis_tmp['tourney_date'].notnull())]
        pl_age = ref.iloc[0]['loser_age']
        tourney_date = ref.iloc[0]['tourney_date']
    
    if pl_age is not None:
        str_tdate = str(tourney_date).replace(".0", "")
        tdate = datetime.strptime(str_tdate, "%Y%m%d")
        decimal_years = pl_age
        decimal_days = decimal_years * 365
        year = decimal_days//365
        tmp_days = decimal_days - (year*365)
        month = tmp_days//31
        days = tmp_days - (month * 31)
        birthdate = tdate - relativedelta(years=year, months=month, days=days)
        
        
        upd_loser = df_tennis_tmp[(df_tennis_tmp['loser_age'].isnull()) & (df_tennis_tmp['loser_name']==pl_name) & (df_tennis_tmp['tourney_date'].notnull())]
        for index, row in upd_loser.iterrows():
            tmp_tourney_date = str(row['tourney_date']).replace(".0", "")
            tmp_tdate = datetime.strptime(str_tdate, "%Y%m%d")
            
            delta = tmp_tdate - birthdate
            days = delta.days/365
            #print(str(days))
            
            df_tennis.loc[
                (df_tennis['loser_age'].isnull()) & 
                (df_tennis['loser_name']==pl_name)  & 
                (df_tennis['tourney_date']==tourney_date), 
                'loser_age'] = days
            
            df_tennis.loc[
                (df_tennis['winner_age'].isnull()) & 
                (df_tennis['winner_name']==pl_name)  & 
                (df_tennis['tourney_date']==tourney_date), 
                'winner_age'] = days
            
        upd_winner = df_tennis_tmp[(df_tennis_tmp['winner_age'].isnull()) & (df_tennis_tmp['winner_name']==pl_name) & (df_tennis_tmp['tourney_date'].notnull())]
        for index, row in upd_winner.iterrows():
            tmp_tourney_date = str(row['tourney_date']).replace(".0", "")
            tmp_tdate = datetime.strptime(str_tdate, "%Y%m%d")
            
            delta = tmp_tdate - birthdate
            days = delta.days/365
            #print(str(days))
            
            df_tennis.loc[
                (df_tennis['loser_age'].isnull()) & 
                (df_tennis['loser_name']==pl_name)  & 
                (df_tennis['tourney_date']==tourney_date), 
                'loser_age'] = days
            
            df_tennis.loc[
                (df_tennis['winner_age'].isnull()) & 
                (df_tennis['winner_name']==pl_name)  & 
                (df_tennis['tourney_date']==tourney_date), 
                'winner_age'] = days
        #break            

In [ ]:
# test print: before was 2851
df_tennis[df_tennis['winner_age'].isnull()][['tourney_date','winner_name','winner_age','loser_name','loser_age']]

In [ ]:
# test print: 6536
df_tennis[df_tennis['loser_age'].isnull()][['tourney_date','winner_name','winner_age','loser_name','loser_age']]

#### winner_age/loser_age, try to calculate birthdate
Convert age in birthdate, based on tourney_date

In [ ]:
for index, row in df_players.iterrows():
      
    age_tdate = df_tennis[
            (df_tennis['winner_name']==row['whole_name']) & 
            (df_tennis['winner_age'].notnull()) & 
            (df_tennis['tourney_date'].notnull())][['winner_age','tourney_date']]
    if age_tdate is None:
        age_tdate = df_tennis[
            (df_tennis['loser_name']==row['whole_name']) & 
            (df_tennis['loser_age'].notnull()) & 
            (df_tennis['tourney_date'].notnull())][['loser_age','tourney_date']]
        
    if age_tdate.size > 0:
        str_tdate = str(age_tdate['tourney_date'].iloc[0]).replace(".0", "")
        
        tdate = datetime.strptime(str_tdate, "%Y%m%d")

        decimal_years = age_tdate['winner_age'].iloc[0]
        decimal_days = decimal_years * 365

        year = decimal_days//365

        tmp_days = decimal_days - (year*365)

        month = tmp_days//31
        days = tmp_days - (month * 31)


        birthdate = tdate - relativedelta(years=year, months=month, days=days)

        strbirthdate = birthdate.strftime("%d/%m/%Y")

        df_players.loc[df_players['whole_name'] == row['whole_name'], 'birthdate'] = strbirthdate

df_players.loc[df_players['birthdate'].isnull(),'birthdate']='unknown' 
df_players


In [ ]:
df_tennis.loc[df_tennis['loser_age'].isnull(),'loser_age'] = -1 
df_tennis.loc[df_tennis['winner_age'].isnull(),'winner_age'] = -1
df_players

#### w_svpt, l_svpt, w_1stIn, l_1stIn, w_1stWon, l_1stWon, w_2ndWon, l_2ndWon, w_df, l_df

Since these values can't be calculated, we substitute null values with a default value. <br>
In the player dataset we calculate for each player: the number of first serves made and won, the number of second serves won and the nuber of double fault.

In [ ]:
w_1stIn_dict = df_tennis[~df_tennis['w_1stIn'].isnull()].groupby('winner_name')['w_1stIn'].apply(list).to_dict()   
for key in w_1stIn_dict.keys():
    w_1stIn_dict[key]= sum(w_1stIn_dict[key])
    
l_1stIn_dict = df_tennis[~df_tennis['l_1stIn'].isnull()].groupby('loser_name')['l_1stIn'].apply(list).to_dict()   
for key in l_1stIn_dict.keys():
    l_1stIn_dict[key]= sum(l_1stIn_dict[key])
    
w_1stwin_dict = df_tennis[~df_tennis['w_1stWon'].isnull()].groupby('winner_name')['w_1stWon'].apply(list).to_dict()   
for key in w_1stwin_dict.keys():
    w_1stwin_dict[key]= sum(w_1stwin_dict[key])
    
l_1stwin_dict = df_tennis[~df_tennis['l_1stWon'].isnull()].groupby('loser_name')['l_1stWon'].apply(list).to_dict()   
for key in l_1stwin_dict.keys():
    l_1stwin_dict[key]= sum(l_1stwin_dict[key])

In [ ]:
#won first services on the total fisrt services not on the total served points
df_players['sv1st'] = 0
df_players['sv1st_win'] = 0

In [ ]:
for index, row in df_players.iterrows():
    name = row['whole_name']
    
    total_first = 0
    won_first = 0
    
    if name in w_1stIn_dict.keys():
        total_first += w_1stIn_dict[name]
        won_first += w_1stwin_dict[name]
    if name in l_1stIn_dict.keys():
        total_first += l_1stIn_dict[name]
        won_first += l_1stwin_dict[name]
        
    if total_first!=0:
        df_players.loc[index, 'sv1st'] = total_first
        df_players.loc[index, 'sv1st_win'] = won_first

In [ ]:
df_players

In [ ]:
w_2ndwin_dict = df_tennis[~df_tennis['w_2ndWon'].isnull()].groupby('winner_name')['w_2ndWon'].apply(list).to_dict()   
for key in w_2ndwin_dict.keys():
    w_2ndwin_dict[key]= sum(w_2ndwin_dict[key])
    
l_2ndwin_dict = df_tennis[~df_tennis['l_2ndWon'].isnull()].groupby('loser_name')['l_2ndWon'].apply(list).to_dict()   
for key in l_2ndwin_dict.keys():
    l_2ndwin_dict[key]= sum(l_2ndwin_dict[key])

In [ ]:
#won second services 
df_players['sv2nd_win'] = 0

In [ ]:
for index, row in df_players.iterrows():
    name = row['whole_name']
    
    won = 0
    
    if name in w_2ndwin_dict.keys():
        won += w_2ndwin_dict[name]
    if name in l_2ndwin_dict.keys():
        won += l_2ndwin_dict[name]
        
    df_players.loc[index, 'sv2nd_win'] = won

In [ ]:
df_players

In [ ]:
w_df_dict = df_tennis[(~df_tennis['w_df'].isnull()) & (~df_tennis['w_svpt'].isnull())].groupby('winner_name')['w_df'].apply(list).to_dict()   
for key in w_df_dict.keys():
    w_df_dict[key]= sum(w_df_dict[key])
    
l_df_dict = df_tennis[(~df_tennis['l_df'].isnull()) & (~df_tennis['l_svpt'].isnull())].groupby('loser_name')['l_df'].apply(list).to_dict()   
for key in l_df_dict.keys():
    l_df_dict[key]= sum(l_df_dict[key])
    
w_sv_dict = df_tennis[(~df_tennis['w_df'].isnull()) & (~df_tennis['w_svpt'].isnull())].groupby('winner_name')['w_svpt'].apply(list).to_dict()   
for key in w_sv_dict.keys():
    w_sv_dict[key]= sum(w_sv_dict[key])
    
l_sv_dict = df_tennis[(~df_tennis['l_df'].isnull()) & (~df_tennis['l_svpt'].isnull())].groupby('loser_name')['l_svpt'].apply(list).to_dict()   
for key in l_sv_dict.keys():
    l_sv_dict[key]= sum(l_sv_dict[key])

In [ ]:
#considered matches
w_match = df_tennis[(~df_tennis['w_df'].isnull()) & (~df_tennis['w_svpt'].isnull())].groupby('winner_name')['w_df'].agg('count').to_dict()
l_match = df_tennis[(~df_tennis['l_df'].isnull()) & (~df_tennis['l_svpt'].isnull())].groupby('loser_name')['l_df'].agg('count').to_dict()

In [ ]:
df_players['df'] = 0

In [ ]:
for index, row in df_players.iterrows():
    name = row['whole_name']
    
    total_points = 0
    df = 0
    if name in w_df_dict.keys():
        total_points += w_sv_dict[name]
        df += w_df_dict[name]
    if name in l_df_dict.keys():
        total_points += l_sv_dict[name]
        df += l_df_dict[name]
        
    df_players.loc[index, 'df'] = df

In [ ]:
df_players[['whole_name', 'sv1st_win', 'df']]

In [ ]:
df_tennis['w_df'].fillna(value=-1, inplace=True) 

In [ ]:
df_tennis['l_df'].fillna(value=-1, inplace=True) 

In [ ]:
df_tennis['w_1stIn'].fillna(value=-1, inplace=True) 

In [ ]:
df_tennis['l_1stIn'].fillna(value=-1, inplace=True) 

In [ ]:
df_tennis['w_1stWon'].fillna(value=-1, inplace=True) 

In [ ]:
df_tennis['l_1stWon'].fillna(value=-1, inplace=True) 

In [ ]:
df_tennis['w_2ndWon'].fillna(value=-1, inplace=True) 

In [ ]:
df_tennis['l_2ndWon'].fillna(value=-1, inplace=True) 

**Service point and ace statistics and cleaning**

In [ ]:
w_svpt = df_tennis[(~df_tennis['w_svpt'].isnull())].groupby('winner_name')['w_svpt'].agg('sum').to_dict()
l_svpt = df_tennis[(~df_tennis['l_svpt'].isnull())].groupby('loser_name')['l_svpt'].agg('sum').to_dict()

#merging players svpts
for l_key in l_svpt.keys():
    if(l_key in w_svpt.keys()):
        l_svpt[l_key]+=w_svpt[l_key]
for w_key in w_svpt.keys():
    if(w_key not in l_svpt.keys()):
        l_svpt[w_key]= w_svpt[w_key]

merged_svp = l_svpt

In [ ]:
df_tennis['w_svpt'].fillna(value=-1, inplace=True) 
df_tennis.loc[df_tennis['l_svpt'] > 300, 'w_svpt'] = -1

In [ ]:
df_tennis['l_svpt'].fillna(value=-1, inplace=True) 
df_tennis.loc[df_tennis['l_svpt'] > 300, 'l_svpt'] = -1

#### w_ace

In the section its computed the total amount of ace performed by each player and added this information to the player dataset.

In [ ]:
w_ace = df_tennis[(~df_tennis['w_ace'].isnull()) & (df_tennis['w_svpt'] != -1)].groupby('winner_name')['w_ace'].agg('sum').to_dict()
l_ace = df_tennis[(~df_tennis['l_ace'].isnull()) & (df_tennis['l_svpt'] != -1)].groupby('loser_name')['l_ace'].agg('sum').to_dict()

#merging players aces
for l_key in l_ace.keys():
    if(l_key in w_ace.keys()):
        l_ace[l_key]+=w_ace[l_key]
for w_key in w_ace.keys():
    if(w_key not in l_ace.keys()):
        l_ace[w_key]= w_ace[w_key]
merged_ace = l_ace
        
for name in merged_ace.keys():
    if (merged_ace[name]):
        svpt = merged_svp[name]
        ace = merged_ace[name]
        df_players.loc[df_players['whole_name'] == name, 'ace'] = ace
df_players.loc[df_players['ace'].isnull(), 'ace'] = 0

In [ ]:
df_players

In [ ]:
df_tennis['w_ace'].fillna(value=-1, inplace=True)

In [ ]:
df_tennis['l_ace'].fillna(value=-1, inplace=True)

#### w_SvGms and l_SvGms

At the end of a set without tie-break, to establish which is in the next set the player that should serve: divide by 2 the sum of all the games played in the set. If the remainder of the division is 0 (even number of games played), the player that serves is the same of the first game of set, otherwise is the other one.

If the set ends with a tie-break (odd number of games already played), the player that serves the next set is not the same of the first game of the set just finished (i.e. the player that serve first the tie-break)

Following these rules we calculate if the number of service games of winner and loser can be valid. If it is not valid we try to correct it looking these rules using the score value. If we cannot calculate the correct value we correct using a default value (-1).

In [ ]:
#correction of errors in case not nulls
for index,row in df_tennis[(~df_tennis['l_SvGms'].isnull()) & (~df_tennis['w_SvGms'].isnull()) & (df_tennis['score']!="")].iterrows():
    
    if len(sets)==1 and (Walkover(sets[0]) or Retirement(sets[0]) or Default(sets[0]) or Bye(sets[0])):
        df_tennis.loc[index, 'w_SvGms']=0
        df_tennis.loc[index, 'l_SvGms']=0
        continue
    
    if not validity_match(row['score'].split(), row['best_of']):
        continue
    
    result = count_service_games(row['score'])
    x = result[0]
    y = result[1]
    if x == -1:
        continue
    
    if not((row['w_SvGms']==x and row['l_SvGms']==y) or (row['w_SvGms']==y and row['l_SvGms']==x)): 
        if row['w_SvGms']==x:
            df_tennis.loc[index, 'l_SvGms']=y
        elif row['w_SvGms']==y:
            df_tennis.loc[index, 'l_SvGms']=x
        elif row['l_SvGms']==x:
            df_tennis.loc[index, 'w_SvGms']=y
        elif row['l_SvGms']==y:
            df_tennis.loc[index, 'w_SvGms']=x
        else:
            df_tennis.loc[index, 'w_SvGms']=x
            df_tennis.loc[index, 'l_SvGms']=y
        

In [ ]:
#correction of errors in case of nulls
for index,row in df_tennis[(df_tennis['score']!="") & (df_tennis['l_SvGms'].isnull())].iterrows():
    
    sets = row['score'].split()
    
    if len(sets)==1 and (Walkover(sets[0]) or Retirement(sets[0]) or Default(sets[0]) or Bye(sets[0])):
        df_tennis.loc[index, 'w_SvGms']=0
        df_tennis.loc[index, 'l_SvGms']=0
        continue
    
    if not validity_match(sets, row['best_of']):
        continue
    
    result = count_service_games(row['score'])
    x = result[0]
    y = result[1]
    if x == -1:
        continue
    
    winner = math.isnan(float(row['w_SvGms']))
    
    if not winner:
        if row['w_SvGms']==x:
            df_tennis.loc[index, 'l_SvGms']=y
        elif row['w_SvGms']==y:
            df_tennis.loc[index, 'l_SvGms']=x
        else:
            df_tennis.loc[index, 'w_SvGms']=x
            df_tennis.loc[index, 'l_SvGms']=y
    else:
        df_tennis.loc[index, 'w_SvGms']=x
        df_tennis.loc[index, 'l_SvGms']=y

In [ ]:
#correction of errors in case of nulls
for index,row in df_tennis[(df_tennis['score']!="") & (df_tennis['w_SvGms'].isnull())].iterrows():
    
    if len(sets)==1 and (Walkover(sets[0]) or Retirement(sets[0]) or Default(sets[0]) or Bye(sets[0])):
        df_tennis.loc[index, 'w_SvGms']=0
        df_tennis.loc[index, 'l_SvGms']=0
        continue
    
    if not validity_match(row['score'].split(), row['best_of']):
        continue
    
    result = count_service_games(row['score'])
    x = result[0]
    y = result[1]
    if x == -1:
        continue
    
    loser = math.isnan(float(row['l_SvGms']))
    
    if not loser:
        if row['l_SvGms']==x:
            df_tennis.loc[index, 'w_SvGms']=y
        elif row['l_SvGms']==y:
            df_tennis.loc[index, 'w_SvGms']=x
        else:
            df_tennis.loc[index, 'w_SvGms']=x
            df_tennis.loc[index, 'l_SvGms']=y
    else:
        df_tennis.loc[index, 'w_SvGms']=x
        df_tennis.loc[index, 'l_SvGms']=y

In [ ]:
df_tennis['l_SvGms'].fillna(value=-1, inplace=True)
df_tennis['w_SvGms'].fillna(value=-1, inplace=True)

**Breakpoint statistics and cleaning**

In the section its computed the total amount of breakpoints saved by each player and added this information to the player dataset.

In [ ]:
df_tennis[df_tennis['w_bpFaced'].isnull()]

In [ ]:
w_bpF = df_tennis[(~df_tennis['w_bpFaced'].isnull()) & (df_tennis['w_bpFaced'] > 0)].groupby('winner_name')['w_bpFaced'].agg('sum').to_dict()
l_bpF = df_tennis[(~df_tennis['l_bpFaced'].isnull()) & (df_tennis['w_bpFaced'] > 0)].groupby('loser_name')['l_bpFaced'].agg('sum').to_dict()

#merging players bpFs
for l_key in l_bpF.keys():
    if(l_key in w_bpF.keys()):
        l_bpF[l_key]+=w_bpF[l_key]
for w_key in w_bpF.keys():
    if(w_key not in l_bpF.keys()):
        l_bpF[w_key]= w_bpF[w_key]
merged_bpF = l_bpF

In [ ]:
df_tennis['w_bpFaced'].fillna(value=-1, inplace=True) 

In [ ]:
df_tennis['l_bpFaced'].fillna(value=-1, inplace=True) 

In [ ]:
df_tennis[df_tennis['w_bpSaved'].isnull()]

In [ ]:
w_bpS = df_tennis[(~df_tennis['w_bpSaved'].isnull()) & (df_tennis['w_bpFaced'] != -1)].groupby('winner_name')['w_bpSaved'].agg('sum').to_dict()
l_bpS = df_tennis[(~df_tennis['l_bpSaved'].isnull()) & (df_tennis['l_bpFaced'] != -1)].groupby('loser_name')['l_bpSaved'].agg('sum').to_dict()

#merging players bpSaveds
for l_key in l_bpS.keys():
    if(l_key in w_bpS.keys()):
        l_bpS[l_key]+=w_bpS[l_key]
        
for w_key in w_bpS.keys():
    if(w_key not in l_bpS.keys()):
        l_bpS[w_key]= w_bpS[w_key]
merged_bpS = l_bpS
              
for name in merged_bpS.keys():
    if (name in l_bpF.keys()):
        bpF = merged_bpF[name]
        bpS = merged_bpS[name]
        df_players.loc[df_players['whole_name'] == name, 'bpS'] = bpS
df_players.loc[df_players['bpS'].isnull(), 'bpS'] = 0

In [ ]:
df_tennis['w_bpSaved'].fillna(value=-1, inplace=True) 

In [ ]:
df_tennis['l_bpSaved'].fillna(value=-1, inplace=True) 

In [ ]:
df_players

#### count the games played, won and lost

In [ ]:
count_dict_w = df_tennis['winner_name'].value_counts().sort_index().to_dict()
count_dict_l = df_tennis['loser_name'].value_counts().sort_index().to_dict()


df_players['wmatch'] = 0
df_players['lmatch'] = 0
df_players['nmatch'] = 0

for index,row in df_players.iterrows():
    name = row['whole_name']
    w = count_dict_w.get(name)
    l = count_dict_l.get(name)
    if w is None:
        w = 0
    if l is None:
        l = 0
    tot = w + l
    df_players.loc[df_players['whole_name'] == name, 'wmatch'] = w
    df_players.loc[df_players['whole_name'] == name, 'lmatch'] = l
    df_players.loc[df_players['whole_name'] == name, 'nmatch'] = tot


In [ ]:
df_players

In [ ]:
df_players.loc[df_players['nmatch']==1]
df_players.loc[df_players['nmatch']!=(df_players['best_of_5_match'] + df_players['best_of_3_match'])]

#### count the win and lose in surfaces

In [ ]:
dict_surf = df_tennis.groupby(['winner_name'])['surface'].value_counts().to_dict()
surface_type_array = df_tennis['surface'].unique()

for index,row in df_players.iterrows():
    name = row['whole_name']
    for surface in surface_type_array:
        win_in_surface = dict_surf.get((name, surface))
        if win_in_surface is None:
            win_in_surface = 0              
        df_players.loc[df_players['whole_name'] == name, "w_surface_" + surface] = str(win_in_surface)
    #if index == 100:
    #    break
# remove unknown w/l columns
df_players.drop(['l_surface_unknown', 'w_surface_unknown'], axis=1)
df_players

In [ ]:
dict_surf = df_tennis.groupby(['loser_name'])['surface'].value_counts().to_dict()
surface_type_array = df_tennis['surface'].unique()

for index,row in df_players.iterrows():
    name = row['whole_name']
    for surface in surface_type_array:
        win_in_surface = dict_surf.get((name, surface))
        if win_in_surface is None:
            win_in_surface = 0              
        df_players.loc[df_players['whole_name'] == name, "l_surface_" + surface] = str(win_in_surface)
    #if index == 100:
    #    break
df_players

In [ ]:
played_touney_winner = df_tennis.groupby('winner_name')['tourney_id'].apply(list).to_dict()  
played_touney_loser = df_tennis.groupby('loser_name')['tourney_id'].apply(list).to_dict()  

In [ ]:
for key in played_touney_winner.keys():
    if key in played_touney_loser.keys():
        played_touney_loser[key] = list(dict.fromkeys(played_touney_winner[key]+played_touney_loser[key]))
    else:
        played_touney_loser[key] = played_touney_winner[key]
        

In [ ]:
played_touney_loser

In [ ]:
df_players['n_tourney'] = 0

In [ ]:
for index,row in df_players.iterrows():
    name = row['whole_name']
    df_players.loc[index, 'n_tourney'] = len(played_touney_loser[name])

In [ ]:
df_players

In [ ]:
#rename column of name of players
df_players.rename(columns={"whole_name": "name"})

In [ ]:
#save on file
df_players.to_csv(path_or_buf="players.csv")

### Correlation

Here we analyze the correlation of the dataset of tennis after all the changes we made.  To do this we do not consider the default values we have introduced because they influence the correlation.

In [ ]:
df_tennis.dtypes

In [ ]:
df_numeric = df_tennis[['draw_size','minutes','w_ace','w_df','w_svpt','w_1stIn', 'w_1stWon', 'w_2ndWon', 
                        'w_SvGms', 'winner_rank', 'winner_rank_points', 'w_bpSaved',  'w_bpFaced','l_ace',
                        'l_df','l_svpt','l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'loser_rank', 
                        'loser_rank_points', 'l_bpSaved',  'l_bpFaced', 'tourney_spectators', 'tourney_revenue',
                       'winner_ht','winner_age','loser_ht','loser_age']]
df_numeric

In [ ]:
plt.figure(figsize = (30,20))
sns.heatmap(df_numeric.corr(), annot=True)

In [ ]:
treshold = 0.85
correlation = df_numeric.corr()
correlation_filtered = correlation[correlation>treshold]
correlation_filtered = correlation[correlation_filtered.sum()>1]
correlation_filtered = correlation_filtered[correlation_filtered>treshold]
correlation_filtered = correlation_filtered.dropna(axis=1,how='all')
correlation_filtered

From https://it.wikipedia.org/wiki/Glossario_del_tennis:
**Breakpoint** - situazione nella quale il giocatore in risposta si trova ad un punto dal vincere il game in cui l'avversario è al servizio.

**w_svpt, w_1stIn, w_1stWon, w_2ndWon** are all correlated because the first serves and the second serve are a part of service points. So if the service points increase the the first services and the second services increase with them.
So more w_svpt increase and more w_1stIn increase. More w_1stIn increase and more is the probability that w_1stWon increase too because more a first serves a player do and more first serves ze can won. The same reasoning is done for w_2ndWon. More service point (w_svpt) a player do and more second serves can ben done and consequences can be won. The same reasoning is done for **l_svpt, l_1stIn, l_1stWon, l_2ndWon**

**w_svpt, w_1stIn, w_1stWon, w_2ndWon, l_svpt, l_1stIn, l_1stWon, l_2ndWon**  The same reasoning of before is done but winner and loser attributes are correlated because the serves are alternated in games, so more services a winner do and more a services a loser do.

**w_bpFaced, w_svpt, w_1stIn, l_svpt, l_1stIn** are all correlated because, the increasing number of service points by the players can be interpreted in two ways: 1. the players make more services in few games reaching many times a draw and this as consequences lead to have many breakpoints because in case of draw we have a breakpoint wvery time a point by a player is done after the draw. 2. The players play many games and to win a game a player must reach the situation where he need only one point to win. So, a breakpoint, to win a game, is present for sure.
So, for these reason, more service points are done by the players and more breakpoints increase. These columns are correlated also to w_1stIn and l_1stIn because to serve a point a players must done a first serve. This reasoning is done also for **l_bpFaced**.

**w_bpSaved, w_bpFaced** are correlated because more are the breakpoints faced and more can be the breakpoint saved.  The same reasoning is done also for the loser  **l_bpSaved, l_bpFaced**.

**w_SvGms, l_SvGms** are correlated because the service is alterned between the winner and loser player by games (the serves approximately the same number of games in a match. So, if w_SvGms increase also l_SvGms increase (cannot have high w_SvGms and low l_SvGms or viceversa because of tennis rules).

**draw_size**, **tourney_revenue**, **tourney_spectators** are all correlated. This can be interpreted as the fact that the tourneys with more players attract more people to see the matches and have a bigger budget respect to the the tourneys with less players.

In [ ]:
correlation_filtered = correlation[correlation<-treshold]
correlation_filtered = correlation_filtered[correlation_filtered<-treshold]
correlation_filtered.dropna(axis=1,how='all',inplace=True)
correlation_filtered